In [1]:
%%time
import pandas as pd
import numpy as np
import h5py
for n,i in enumerate([0,1,2,4,5,7,8,10,11,14,15,16,18,19,20,21,23,25,26,28,29,30,31,32,34,36,37,38,40,43,44,45,47,48,50]):
    if n==0:
        with h5py.File(f'D:\Projects\Goods_prediction\\real_data\Dat9\data{i}.h5', 'r') as hf:
            data = hf['data'][:]
        train_data=data[:,:400]
        train_label=data[:,400:]
    else:
        with h5py.File(f'D:\Projects\Goods_prediction\\real_data\Dat9\data{i}.h5', 'r') as hf:
            data = hf['data'][:]
        train_data=np.concatenate([train_data,data[:,:400]])
        train_label=np.concatenate([train_label,data[:,400:]])
    print(f'{n} train files loaded')
del data
print('Loaded train data')

for n,i in enumerate([3,6,9,12,13,17,22,24,27,33,35,39,41,42,46,49,51]):
    if n==0:
        with h5py.File(f'D:\Projects\Goods_prediction\\real_data\Dat9\data{i}.h5', 'r') as hf:
            data = hf['data'][:]
        val_data=data[:,:400]
        val_label=data[:,400:]
    else:
        with h5py.File(f'D:\Projects\Goods_prediction\\real_data\Dat9\data{i}.h5', 'r') as hf:
            data = hf['data'][:]
        val_data=np.concatenate([val_data,data[:,:400]])
        val_label=np.concatenate([val_label,data[:,400:]])
    print(f'{n} test files loaded')
import gc
gc.collect()

0 train files loaded
1 train files loaded
2 train files loaded
3 train files loaded
4 train files loaded
5 train files loaded
6 train files loaded
7 train files loaded
8 train files loaded
9 train files loaded
10 train files loaded
11 train files loaded
12 train files loaded
13 train files loaded
14 train files loaded
15 train files loaded
16 train files loaded
17 train files loaded
18 train files loaded
19 train files loaded
20 train files loaded
21 train files loaded
22 train files loaded
23 train files loaded
24 train files loaded
25 train files loaded
26 train files loaded
27 train files loaded
28 train files loaded
29 train files loaded
30 train files loaded
31 train files loaded
32 train files loaded
33 train files loaded
34 train files loaded
Loaded train data
0 test files loaded
1 test files loaded
2 test files loaded
3 test files loaded
4 test files loaded
5 test files loaded
6 test files loaded
7 test files loaded
8 test files loaded
9 test files loaded
10 test files loaded
1

0

In [4]:
%%time
# import pickle

# with open(r"D:\Projects\Goods_prediction\train_data.pkl","wb") as f:
#     pickle.dump(train_data,f, protocol=4)
# with open(r"D:\Projects\Goods_prediction\train_target.pkl","wb") as f:
#     pickle.dump(train_target,f, protocol=4) 
# with open(r"D:\Projects\Goods_prediction\test_data.pkl","wb") as f:
#     pickle.dump(test_data,f, protocol=4)
# with open(r"D:\Projects\Goods_prediction\test_target.pkl","wb") as f:
#     pickle.dump(test_target,f, protocol=4) 
train_label

Wall time: 0 ns


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [2]:
import keras
from keras.models import Sequential,Model
from keras import backend as K
from keras.layers import Dense,Dropout,Input,BatchNormalization,LSTM
from keras.layers import GRU,Bidirectional,GlobalMaxPooling1D,Conv1D,Flatten,Embedding,GlobalAveragePooling1D
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint,Callback,ReduceLROnPlateau,EarlyStopping
from keras.metrics import top_k_categorical_accuracy
from sklearn.preprocessing import Normalizer
from joblib import Parallel, delayed
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
import multiprocessing
import tensorflow as tf
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
K.set_session(tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=16, inter_op_parallelism_threads=16)))
K.clear_session()
def accur(y_true, y_pred,limit):
#     itog_pred=K.greater(pred>limit)
#     itog_true=K.equal(y_true,1)
    score=[]
    for true,pred in zip(y_true, y_pred):
        itog_pred=np.squeeze((pred>limit).nonzero()[0])
        itog_true=np.squeeze(np.argwhere(true==1))
        try:
            score.append(sum([np.any(i==itog_true[:]) for i in itog_pred])/len(itog_pred))
        except ZeroDivisionError:
            pass
        except IndexError:
            score.append(sum([np.any(i==itog_true) for i in itog_pred])/len(itog_pred))
        except TypeError:
            pass
    return np.mean(score)
def accur2(y_true, y_pred,limit):
#     itog_pred=K.greater(pred>limit)
#     itog_true=K.equal(y_true,1)
    score=[]
    score_null=[]
    for true,pred in zip(y_true, y_pred):
        itog_pred=np.squeeze((pred>limit).nonzero()[0])
        itog_true=np.squeeze(np.argwhere(true==1))
        if itog_pred.size>0:
            try:
                score.append(int(sum([np.any(i==itog_true[:]) for i in itog_pred])>0))
            except IndexError:
                score.append(int(sum([np.any(i==itog_true) for i in itog_pred])>0))
            except TypeError:
                pass
        else:
            score_null.append(int(itog_true.size==0))
    return np.mean(score),len(score)/(len(score)+len(score_null))
def top_category(y_true, y_pred):
    result=[]
    result=Parallel(n_jobs=-2)(delayed(accur2)(y_true, y_pred,l) for l in [0.1,0.3,0.5])
#    for l in [0.1,0.29]:
#        result.append(accur2(y_true,y_pred,limit=l))
    return result
def my_accur(y_true, y_pred):
    result=[]
    for l in np.arange(0.3,0.95,0.01):
        prediction=K.cast(K.greater(y_pred,l),K.floatx())
        result.append(K.mean(K.equal(y_true,prediction)))
 #   print(np.argmax(result)*0.01+0.1)
    return K.max(result)
from sklearn.metrics import fbeta_score
def fbeta_predict(prediction,labels):
    score = []
    candidates = np.arange(0, 0.5, 0.01)
    for th in candidates:
        yp = (prediction > th).astype(int)
        score.append(fbeta_score(y_pred=yp, y_true=labels, beta=2, average="samples"))
    score = np.array(score)
    pm = score.argmax()
    best_th, best_score = candidates[pm], score[pm]
    print(f"Best limit {best_th}")
    return best_score
gamma = 2.0
epsilon = K.epsilon()
def focal_loss(y_true, y_pred):
    pt = y_pred * y_true + (1-y_pred) * (1-y_true)
    pt = K.clip(pt, epsilon, 1-epsilon)
    CE = -K.log(pt)
    FL = K.pow(1-pt, gamma) * CE
    loss = K.sum(FL, axis=1)
    return loss
def accur2(y_true, y_pred,limit):
#     itog_pred=K.greater(pred>limit)
#     itog_true=K.equal(y_true,1)
    score=[]
    score_null=[]
    itog_num=[]
    for true,pred in zip(y_true, y_pred):
        itog_pred=np.squeeze((pred>limit).nonzero()[0])
        itog_true=np.squeeze(np.argwhere(true==1))
        if itog_pred.size>0:
            try:
                score.append(sum([np.any(i==itog_true[:]) for i in itog_pred])/len(itog_pred))
                itog_num.append(len(itog_pred))
            except IndexError:
                score.append(sum([np.any(i==itog_true) for i in itog_pred])/len(itog_pred))
                itog_num.append(len(itog_pred))
            except TypeError:
                pass
        else:
            score_null.append(int(itog_true.size==0))
    return np.mean(score),len(score)/(len(score)+len(score_null)),np.mean(itog_num)
def top_category(y_true, y_pred):
    result=[]
 #   result=Parallel(n_jobs=-2)(delayed(accur2)(y_true, y_pred,l) for l in [0.3,0.4,0.5])
    for l in [0.3,0.4,0.5]:
        result.append(accur2(y_true,y_pred,limit=l))
    return result

# train_data=norm.transform(train[df_train_columns])
# train_label=np.asarray(train_target)
# val_data=norm.transform(test[df_train_columns])
# val_label=np.asarray(test_target)
#train=train_data.copy()
#train_data=train[ind]
inp=Input(shape=(train_data.shape[1],))
emb=Embedding(5656,70)(inp)
#x=Bidirectional(LSTM(8,return_sequences=True))(emb)
#x=Bidirectional(LSTM(8,return_sequences=True))(x)
#x=Attention(train_data.shape[1])(x)
x=Flatten()(emb)

x=Dense(1925,activation='sigmoid')(x)
model=Model(inputs=inp,outputs=x)
class Metr(Callback):
    def __init__(self,N):
        self.N=N
#        self.epoch=1
    def on_epoch_end(self,epoch, logs=None):
        if epoch%self.N==1:
            val_pr=np.squeeze(model.predict([val_data], batch_size=2024, verbose=1))
            result=top_category(val_label,val_pr)
            print('0.3 limit top5_accuracy: {} all_accuracy {} mean_goods_number {}'.format(result[0][0],result[0][1],result[0][2]))
            print('0.4 limit top5_accuracy: {} all_accuracy {} mean_goods_number {}'.format(result[1][0],result[1][1],result[1][2]))
            print('0.5 limit top5_accuracy: {} all_accuracy {} mean_goods_number {}'.format(result[2][0],result[2][1],result[2][2]))
metr=Metr(10)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=7, min_lr=1e-6, mode='auto', verbose=1)
earlstop=EarlyStopping(patience=3)
checkpointer = ModelCheckpoint(filepath='D:\Projects\Goods_prediction\\real_data\\model_rnn_1l.hdf5', verbose=1, save_best_only=True)
model.compile(optimizer=adam(), loss=focal_loss)
model.fit(train_data,train_label,batch_size=8000,epochs=25,validation_data=(val_data,val_label),verbose=1,callbacks=[earlstop,checkpointer,metr])
model.load_weights('D:\Projects\Goods_prediction\\real_data\model_rnn_1l.hdf5')
model.save("model_rnn_1l.h5")
val_pr=np.squeeze(model.predict([val_data], batch_size=1024, verbose=1))
#0.6830883802939437, 0.9055531263664189
#0.79862720237835, 0.572865057476875
#0.1268

Using TensorFlow backend.
W1007 22:30:03.019440 15560 deprecation_wrapper.py:119] From C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:107: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W1007 22:30:03.019440 15560 deprecation_wrapper.py:119] From C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:111: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1007 22:30:03.029454 15560 deprecation_wrapper.py:119] From C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1007 22:30:03.099398 15560 deprecation_wrapper.py:119] From C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum

Train on 5674557 samples, validate on 2705070 samples
Epoch 1/25
5674557/5674557 [==============================] - 3161s 557us/step - loss: 16.7409 - val_loss: 14.7284

Epoch 00001: val_loss improved from inf to 14.72843, saving model to D:\Projects\Goods_prediction\real_data\model_rnn_1l.hdf5
Epoch 2/25
5674557/5674557 [==============================] - 3162s 557us/step - loss: 14.7678 - val_loss: 14.6044

Epoch 00002: val_loss improved from 14.72843 to 14.60435, saving model to D:\Projects\Goods_prediction\real_data\model_rnn_1l.hdf5
2705070/2705070 [==============================] - 406s 150us/step
0.3 limit top5_accuracy: 0.20566633808380708 all_accuracy 1.0 mean_goods_number 35.540956426266234
0.4 limit top5_accuracy: 0.4062816340850344 all_accuracy 0.9446868144377253 mean_goods_number 9.440135417766621
0.5 limit top5_accuracy: 0.6036085559358126 all_accuracy 0.5651103829673669 mean_goods_number 4.1842466876557785
Epoch 3/25
5674557/5674557 [==============================] - 3161

In [24]:
%%time
from keras.models import load_model
model=load_model('D:\Projects\Goods_prediction\\real_data\\model_rnn_mDense_2l.hdf5')
val_data=test_data
val_label=test_target
#val_pr=np.squeeze(model.predict([val_data], batch_size=1024, verbose=0))
goods=pd.read_csv('D:\Projects\Goods_prediction\\real_data\goods2id.csv',sep=';',names=['GoodsGroup3Id','GoodsGroup3name'])
goods=goods.reset_index(drop=False)
goods=goods.rename(columns={'index':'ind'})
goods=goods['GoodsGroup3name'].values
def category_name(y_true, y_pred,limit):
#     itog_pred=K.greater(pred>limit)
#     itog_true=K.equal(y_true,1)
    itog_pred=[]
    itog_true=[]
    for true,pred in zip(y_true, y_pred):
#        print(f'pred {[goods[i] for i in [np.squeeze((pred>limit).nonzero()[0])]]}')
#        print(f'true {[goods[i] for i in [np.squeeze(np.argwhere(true==1))]]}')

        itog_pred.append(np.squeeze([goods[i] for i in [np.squeeze((pred>limit).nonzero()[0])]]))
        if len(np.argwhere(true==1))==1:
            itog_true.append(np.squeeze([[goods[i],pred[i]] for i in [np.squeeze(np.argwhere(true==1))]]))
        else:
            itog_true.append(np.squeeze([list(zip(goods[i],pred[i])) for i in [np.squeeze(np.argwhere(true==1))]]))            
    return itog_true,itog_pred
true_label,predict_label=category_name(val_label,val_pr,0.5)
pd.DataFrame({'true_label':true_label,'predict_label':predict_label}).to_csv('resultrnn2id.csv',sep=';',encoding='cp1251')

Wall time: 6min 31s


In [9]:
%%time
#ind=np.random.randint(0,len(train_data),size=1000000)
#val_data=test_data
from keras.models import load_model
#model=load_model('D:\Projects\Goods_prediction\\real_data\\model_rnn_mDense_2l.hdf5')
#model.load_weights('D:\Projects\Goods_prediction\\real_data\model_weights_rnn.hdf5')
#val_pr=np.squeeze(model.predict([val_data], batch_size=2048, verbose=0))
def test():
    ttest=pd.read_hdf('D:\Projects\Goods_prediction\\real_data\\Dat3\\test.h5','data')['goods'][0]
#    ttest=ttest[df_train_columns].values
#    norm.transform(ttest,copy=False)
    pred=np.squeeze(model.predict([[ttest]], batch_size=1024, verbose=1))
    return np.squeeze([goods[i] for i in [np.squeeze((pred>0.4).nonzero()[0])]]) 

def accur2(y_true, y_pred,limit):
#     itog_pred=K.greater(pred>limit)
#     itog_true=K.equal(y_true,1)
    score=[]
    score_null=[]
    itog_num=[]
    for true,pred in zip(y_true, y_pred):
        itog_pred=np.squeeze((pred>limit).nonzero()[0])
        itog_true=np.squeeze(np.argwhere(true==1))
        if itog_pred.size>0:
            try:
                score.append(sum([np.any(i==itog_true[:]) for i in itog_pred])/len(itog_pred))
                itog_num.append(len(itog_pred))
            except IndexError:
                score.append(sum([np.any(i==itog_true) for i in itog_pred])/len(itog_pred))
                itog_num.append(len(itog_pred))
            except TypeError:
                pass
        else:
            score_null.append(int(itog_true.size==0))
    return np.mean(score),len(score)/(len(score)+len(score_null)),np.mean(itog_num)
def top_category(y_true, y_pred):
    result=[]
#    result=Parallel(n_jobs=3)(delayed(accur2)(y_true, y_pred,l) for l in [0.3,0.4,0.5])
    for l in [0.52,0.54,0.56]:
        result.append(accur2(y_true,y_pred,limit=l))
    return result
print(top_category(val_label,val_pr))

[(0.6411636542956115, 0.7387811576277227, 4.278121131750185), (0.6529116224042888, 0.7056164408060762, 4.118380515278466), (0.6645067154495161, 0.6714930687428788, 3.9675537830353225)]
Wall time: 2min 56s


In [14]:
#top_category(val_label,val_pr)
isinstance(292,int)

True

In [35]:
goods=pd.read_csv('D:\Projects\Goods_prediction\\real_data\goods2id.csv',sep=';',names=['GoodsGroup3Id','GoodsGroup3name'])
goods=goods.reset_index(drop=False)
goods=goods.rename(columns={'index':'ind'})
goods
goods[goods['ind']==623]

,ind,GoodsGroup3Id,GoodsGroup3name
623,623,2563,Корма для грызунов


In [33]:
#top_category(test_target,val_pr)
mm.most_similar('52')

C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('333', 0.547301173210144),
 ('623', 0.5419012308120728),
 ('187', 0.5062810182571411),
 ('368', 0.4683801531791687),
 ('49', 0.4316135048866272),
 ('580', 0.41029298305511475),
 ('371', 0.40346166491508484),
 ('276', 0.40021929144859314),
 ('512', 0.3969462215900421),
 ('289', 0.3889583647251129)]

In [ ]:
import keras
from keras.models import Sequential,Model
from keras import backend as K
from keras.layers import Dense,Dropout,Input,BatchNormalization,LSTM
from keras.layers import GRU,Bidirectional,GlobalMaxPooling1D,Conv1D,Flatten,Embedding,GlobalAveragePooling1D
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint,Callback,ReduceLROnPlateau,EarlyStopping
from keras.metrics import top_k_categorical_accuracy
from sklearn.preprocessing import Normalizer
from joblib import Parallel, delayed
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
import multiprocessing
import tensorflow as tf
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
K.set_session(tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=16, inter_op_parallelism_threads=16)))
K.clear_session()
def accur(y_true, y_pred,limit):
#     itog_pred=K.greater(pred>limit)
#     itog_true=K.equal(y_true,1)
    score=[]
    for true,pred in zip(y_true, y_pred):
        itog_pred=np.squeeze((pred>limit).nonzero()[0])
        itog_true=np.squeeze(np.argwhere(true==1))
        try:
            score.append(sum([np.any(i==itog_true[:]) for i in itog_pred])/len(itog_pred))
        except ZeroDivisionError:
            pass
        except IndexError:
            score.append(sum([np.any(i==itog_true) for i in itog_pred])/len(itog_pred))
        except TypeError:
            pass
    return np.mean(score)
def accur2(y_true, y_pred,limit):
#     itog_pred=K.greater(pred>limit)
#     itog_true=K.equal(y_true,1)
    score=[]
    score_null=[]
    for true,pred in zip(y_true, y_pred):
        itog_pred=np.squeeze((pred>limit).nonzero()[0])
        itog_true=np.squeeze(np.argwhere(true==1))
        if itog_pred.size>0:
            try:
                score.append(int(sum([np.any(i==itog_true[:]) for i in itog_pred])>0))
            except IndexError:
                score.append(int(sum([np.any(i==itog_true) for i in itog_pred])>0))
            except TypeError:
                pass
        else:
            score_null.append(int(itog_true.size==0))
    return np.mean(score),len(score)/(len(score)+len(score_null))
def top_category(y_true, y_pred):
    result=[]
    result=Parallel(n_jobs=-2)(delayed(accur2)(y_true, y_pred,l) for l in [0.1,0.3,0.5])
#    for l in [0.1,0.29]:
#        result.append(accur2(y_true,y_pred,limit=l))
    return result
def my_accur(y_true, y_pred):
    result=[]
    for l in np.arange(0.3,0.95,0.01):
        prediction=K.cast(K.greater(y_pred,l),K.floatx())
        result.append(K.mean(K.equal(y_true,prediction)))
 #   print(np.argmax(result)*0.01+0.1)
    return K.max(result)
from sklearn.metrics import fbeta_score
def fbeta_predict(prediction,labels):
    score = []
    candidates = np.arange(0, 0.5, 0.01)
    for th in candidates:
        yp = (prediction > th).astype(int)
        score.append(fbeta_score(y_pred=yp, y_true=labels, beta=2, average="samples"))
    score = np.array(score)
    pm = score.argmax()
    best_th, best_score = candidates[pm], score[pm]
    print(f"Best limit {best_th}")
    return best_score
gamma = 2.0
epsilon = K.epsilon()
def focal_loss(y_true, y_pred):
    pt = y_pred * y_true + (1-y_pred) * (1-y_true)
    pt = K.clip(pt, epsilon, 1-epsilon)
    CE = -K.log(pt)
    FL = K.pow(1-pt, gamma) * CE
    loss = K.sum(FL, axis=1)
    return loss
def accur2(y_true, y_pred,limit):
#     itog_pred=K.greater(pred>limit)
#     itog_true=K.equal(y_true,1)
    score=[]
    score_null=[]
    itog_num=[]
    for true,pred in zip(y_true, y_pred):
        itog_pred=np.squeeze((pred>limit).nonzero()[0])
        itog_true=np.squeeze(np.argwhere(true==1))
        if itog_pred.size>0:
            try:
                score.append(sum([np.any(i==itog_true[:]) for i in itog_pred])/len(itog_pred))
                itog_num.append(len(itog_pred))
            except IndexError:
                score.append(sum([np.any(i==itog_true) for i in itog_pred])/len(itog_pred))
                itog_num.append(len(itog_pred))
            except TypeError:
                pass
        else:
            score_null.append(int(itog_true.size==0))
    return np.mean(score),len(score)/(len(score)+len(score_null)),np.mean(itog_num)
def top_category(y_true, y_pred):
    result=[]
 #   result=Parallel(n_jobs=-2)(delayed(accur2)(y_true, y_pred,l) for l in [0.3,0.4,0.5])
    for l in [0.3,0.4,0.5]:
        result.append(accur2(y_true,y_pred,limit=l))
    return result
val_pr=np.zeros(len(test_data))
# train_data=norm.transform(train[df_train_columns])
# train_label=np.asarray(train_target)
# val_data=norm.transform(test[df_train_columns])
# val_label=np.asarray(test_target)
#train=train_data.copy()
#train_data=train[ind]
train_label=train_target
val_data=test_data
val_label=test_target
inp=Input(shape=(train_data.shape[1],))
emb=Embedding(5656,50)(inp)
#x=Bidirectional(LSTM(8,return_sequences=True))(emb)
#x=Bidirectional(LSTM(8,return_sequences=True))(x)
#x=Attention(train_data.shape[1])(x)
x=Flatten()(emb)

x=Dense(788,activation='sigmoid')(x)
model=Model(inputs=inp,outputs=x)
class Metr(Callback):
    def __init__(self,N):
        self.N=N
#        self.epoch=1
    def on_epoch_end(self,epoch, logs=None):
        if epoch%self.N==0:
            val_pr=np.squeeze(model.predict([val_data], batch_size=1024, verbose=1))
            result=top_category(val_label,val_pr)
            print('0.3 limit top5_accuracy: {} all_accuracy {} mean_goods_number {}'.format(result[0][0],result[0][1],result[0][2]))
            print('0.4 limit top5_accuracy: {} all_accuracy {} mean_goods_number {}'.format(result[1][0],result[1][1],result[1][2]))
            print('0.5 limit top5_accuracy: {} all_accuracy {} mean_goods_number {}'.format(result[2][0],result[2][1],result[2][2]))
metr=Metr(10)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=7, min_lr=1e-6, mode='auto', verbose=1)
earlstop=EarlyStopping(patience=3)
checkpointer = ModelCheckpoint(filepath='D:\Projects\Goods_prediction\\real_data\\model_50rnn_1l.hdf5', verbose=1, save_best_only=True)
model.compile(optimizer=adam(), loss='binary_crossentropy')
model.fit(train_data,train_label,batch_size=8000,epochs=25,validation_data=(val_data,val_label),verbose=1,callbacks=[earlstop,checkpointer,metr])
model.load_weights('D:\Projects\Goods_prediction\\real_data\model_50rnn_1l.hdf5')
model.save("model_50rnn_1l.h5")
val_pr=np.squeeze(model.predict([val_data], batch_size=1024, verbose=1))
#0.6830883802939437, 0.9055531263664189
#0.79862720237835, 0.572865057476875
#0.1268

In [ ]:
import keras
from keras.models import Sequential,Model
from keras import backend as K
from keras.layers import Dense,Dropout,Input,BatchNormalization,LSTM
from keras.layers import GRU,Bidirectional,GlobalMaxPooling1D,Conv1D,Flatten,Embedding,GlobalAveragePooling1D
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint,Callback,ReduceLROnPlateau,EarlyStopping
from keras.metrics import top_k_categorical_accuracy
from sklearn.preprocessing import Normalizer
from joblib import Parallel, delayed
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
import multiprocessing
import tensorflow as tf
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
K.set_session(tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=16, inter_op_parallelism_threads=16)))
K.clear_session()
def accur(y_true, y_pred,limit):
#     itog_pred=K.greater(pred>limit)
#     itog_true=K.equal(y_true,1)
    score=[]
    for true,pred in zip(y_true, y_pred):
        itog_pred=np.squeeze((pred>limit).nonzero()[0])
        itog_true=np.squeeze(np.argwhere(true==1))
        try:
            score.append(sum([np.any(i==itog_true[:]) for i in itog_pred])/len(itog_pred))
        except ZeroDivisionError:
            pass
        except IndexError:
            score.append(sum([np.any(i==itog_true) for i in itog_pred])/len(itog_pred))
        except TypeError:
            pass
    return np.mean(score)
def accur2(y_true, y_pred,limit):
#     itog_pred=K.greater(pred>limit)
#     itog_true=K.equal(y_true,1)
    score=[]
    score_null=[]
    for true,pred in zip(y_true, y_pred):
        itog_pred=np.squeeze((pred>limit).nonzero()[0])
        itog_true=np.squeeze(np.argwhere(true==1))
        if itog_pred.size>0:
            try:
                score.append(int(sum([np.any(i==itog_true[:]) for i in itog_pred])>0))
            except IndexError:
                score.append(int(sum([np.any(i==itog_true) for i in itog_pred])>0))
            except TypeError:
                pass
        else:
            score_null.append(int(itog_true.size==0))
    return np.mean(score),len(score)/(len(score)+len(score_null))
def top_category(y_true, y_pred):
    result=[]
    result=Parallel(n_jobs=-2)(delayed(accur2)(y_true, y_pred,l) for l in [0.1,0.3,0.5])
#    for l in [0.1,0.29]:
#        result.append(accur2(y_true,y_pred,limit=l))
    return result
def my_accur(y_true, y_pred):
    result=[]
    for l in np.arange(0.3,0.95,0.01):
        prediction=K.cast(K.greater(y_pred,l),K.floatx())
        result.append(K.mean(K.equal(y_true,prediction)))
 #   print(np.argmax(result)*0.01+0.1)
    return K.max(result)
from sklearn.metrics import fbeta_score
def fbeta_predict(prediction,labels):
    score = []
    candidates = np.arange(0, 0.5, 0.01)
    for th in candidates:
        yp = (prediction > th).astype(int)
        score.append(fbeta_score(y_pred=yp, y_true=labels, beta=2, average="samples"))
    score = np.array(score)
    pm = score.argmax()
    best_th, best_score = candidates[pm], score[pm]
    print(f"Best limit {best_th}")
    return best_score
gamma = 2.0
epsilon = K.epsilon()
def focal_loss(y_true, y_pred):
    pt = y_pred * y_true + (1-y_pred) * (1-y_true)
    pt = K.clip(pt, epsilon, 1-epsilon)
    CE = -K.log(pt)
    FL = K.pow(1-pt, gamma) * CE
    loss = K.sum(FL, axis=1)
    return loss
def accur2(y_true, y_pred,limit):
#     itog_pred=K.greater(pred>limit)
#     itog_true=K.equal(y_true,1)
    score=[]
    score_null=[]
    itog_num=[]
    for true,pred in zip(y_true, y_pred):
        itog_pred=np.squeeze((pred>limit).nonzero()[0])
        itog_true=np.squeeze(np.argwhere(true==1))
        if itog_pred.size>0:
            try:
                score.append(sum([np.any(i==itog_true[:]) for i in itog_pred])/len(itog_pred))
                itog_num.append(len(itog_pred))
            except IndexError:
                score.append(sum([np.any(i==itog_true) for i in itog_pred])/len(itog_pred))
                itog_num.append(len(itog_pred))
            except TypeError:
                pass
        else:
            score_null.append(int(itog_true.size==0))
    return np.mean(score),len(score)/(len(score)+len(score_null)),np.mean(itog_num)
def top_category(y_true, y_pred):
    result=[]
 #   result=Parallel(n_jobs=-2)(delayed(accur2)(y_true, y_pred,l) for l in [0.3,0.4,0.5])
    for l in [0.3,0.4,0.5]:
        result.append(accur2(y_true,y_pred,limit=l))
    return result
val_pr=np.zeros(len(test_data))
# train_data=norm.transform(train[df_train_columns])
# train_label=np.asarray(train_target)
# val_data=norm.transform(test[df_train_columns])
# val_label=np.asarray(test_target)
#train=train_data.copy()
#train_data=train[ind]
train_label=train_target
val_data=test_data
val_label=test_target
inp=Input(shape=(train_data.shape[1],))
emb=Embedding(5656,90)(inp)
#x=Bidirectional(LSTM(8,return_sequences=True))(emb)
#x=Bidirectional(LSTM(8,return_sequences=True))(x)
#x=Attention(train_data.shape[1])(x)
x=Flatten()(emb)

x=Dense(788,activation='sigmoid')(x)
model=Model(inputs=inp,outputs=x)
class Metr(Callback):
    def __init__(self,N):
        self.N=N
#        self.epoch=1
    def on_epoch_end(self,epoch, logs=None):
        if epoch%self.N==0:
            val_pr=np.squeeze(model.predict([val_data], batch_size=1024, verbose=1))
            result=top_category(val_label,val_pr)
            print('0.3 limit top5_accuracy: {} all_accuracy {} mean_goods_number {}'.format(result[0][0],result[0][1],result[0][2]))
            print('0.4 limit top5_accuracy: {} all_accuracy {} mean_goods_number {}'.format(result[1][0],result[1][1],result[1][2]))
            print('0.5 limit top5_accuracy: {} all_accuracy {} mean_goods_number {}'.format(result[2][0],result[2][1],result[2][2]))
metr=Metr(10)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=7, min_lr=1e-6, mode='auto', verbose=1)
earlstop=EarlyStopping(patience=3)
checkpointer = ModelCheckpoint(filepath='D:\Projects\Goods_prediction\\real_data\\model_90rnn_1l.hdf5', verbose=1, save_best_only=True)
model.compile(optimizer=adam(), loss='binary_crossentropy')
model.fit(train_data,train_label,batch_size=8000,epochs=25,validation_data=(val_data,val_label),verbose=1,callbacks=[earlstop,checkpointer,metr])
model.load_weights('D:\Projects\Goods_prediction\\real_data\model_90rnn_1l.hdf5')
model.save("model_90rnn_1l.h5")
val_pr=np.squeeze(model.predict([val_data], batch_size=1024, verbose=1))
#0.6830883802939437, 0.9055531263664189
#0.79862720237835, 0.572865057476875
#0.1268

In [1]:
import pandas as pd
a=pd.read_csv('resultrnn2id.csv')
a

ParserError: Error tokenizing data. C error: Expected 1 fields in line 13, saw 2
